<a href="https://colab.research.google.com/github/dmnarula/football-ds-learning/blob/main/week4-modeling/Week4_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
%cd /content
!rm -rf football-ds-learning
!git clone https://github.com/dmnarula/football-ds-learning.git

/content
Cloning into 'football-ds-learning'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 160 (delta 82), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 1.24 MiB | 5.39 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [54]:
%cd football-ds-learning
!ls

/content/football-ds-learning
README.md    src		  week2-pandas	       week4-modeling
sample_data  week1-python-basics  week3-visualization


In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot  as plt
import os
import sys
sys.path.append("../..")
from src.merging import build_match_df
from sklearn.linear_model import LinearRegression

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

In [56]:
base_gw_url = ("https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/refs/heads/main/data/2025-2026/By%20Gameweek")

# Automate GW URLs
def get_gw_urls(gw: int):
  gw_path = f"{base_gw_url}/GW{gw}"
  url_stats = f"{gw_path}/playermatchstats.csv"
  url_players = f"{gw_path}/players.csv"
  url_teams = f"{gw_path}/teams.csv"
  url_matches = f"{gw_path}/matches.csv"

  return url_stats, url_players, url_teams, url_matches

In [57]:

# Season data loader
def build_season_df(start_gw, end_gw):
  all_gw_dfs = []

  for gw in range(start_gw, end_gw + 1):
    url_stats, url_players, url_teams, url_matches = get_gw_urls(gw)

    df_gw = build_match_df(
        url_stats=url_stats,
        url_players=url_players,
        url_teams=url_teams,
        url_matches=url_matches
    )

    df_gw["gameweek"]=gw
    all_gw_dfs.append(df_gw)

  return pd.concat(all_gw_dfs, ignore_index=True)

In [58]:
df_season = build_season_df(1, 15)
df_season.head()


,player_id,match_id,minutes_played,goals,assists,total_shots,xg,xa,shots_on_target,successful_dribbles,...,home_team,away_team,home_score,away_score,home_team_name,home_team_short_name,away_team_name,away_team_short_name,team_name,team_short_name
0,443,25-26-prem-manchester-united-vs-arsenal,80,0,0,0,0.00,0.02,0,0,...,1,3,0.0,1.0,Man Utd,MUN,Arsenal,ARS,Man Utd,MUN
1,16,25-26-prem-manchester-united-vs-arsenal,90,0,0,1,0.15,0.09,0,3,...,1,3,0.0,1.0,Man Utd,MUN,Arsenal,ARS,Arsenal,ARS
2,525,25-26-prem-nottingham-forest-vs-brentford,79,2,0,2,0.92,0.01,2,1,...,17,94,3.0,1.0,Nott'm Forest,NFO,Brentford,BRE,Nott'm Forest,NFO
3,515,25-26-prem-nottingham-forest-vs-brentford,84,0,1,3,0.38,0.28,0,2,...,17,94,3.0,1.0,Nott'm Forest,NFO,Brentford,BRE,Nott'm Forest,NFO
4,517,25-26-prem-nottingham-forest-vs-brentford,90,0,1,3,0.39,0.28,1,0,...,17,94,3.0,1.0,Nott'm Forest,NFO,Brentford,BRE,Nott'm Forest,NFO
